# CS 3110/5110: Data Privacy
## In-Class Exercises, week of 9/4/2023

In [1]:
# Load the data and libraries
import pandas as pd
import numpy as np

adult = pd.read_csv('https://github.com/jnear/cs211-data-privacy/raw/master/homework/adult_with_pii.csv')

## Question 1

Write code to group the `adult` dataset by a single column and count the number of members in each group.

In [2]:
def group_one_count(col):
    return adult.groupby(col).size()

In [3]:
group_one_count('Education')

Education
10th              933
11th             1175
12th              433
1st-4th           168
5th-6th           333
7th-8th           646
9th               514
Assoc-acdm       1067
Assoc-voc        1382
Bachelors        5355
Doctorate         413
HS-grad         10501
Masters          1723
Preschool          51
Prof-school       576
Some-college     7291
dtype: int64

In [4]:
s = group_one_count('Education')
assert s['10th'] == 933
assert s['9th'] == 514
assert s['Some-college'] == 7291

## Question 2

Write code to group the `adult` dataset by two columns and count the number of members in each group.

In [5]:
def group_two_count(col1, col2):
    return adult.groupby([col1, col2]).size()

In [6]:
group_two_count('Occupation', 'Education')

Occupation        Education   
Adm-clerical      10th             38
                  11th             67
                  12th             38
                  5th-6th           6
                  7th-8th          11
                                 ... 
Transport-moving  Doctorate         1
                  HS-grad         825
                  Masters          10
                  Prof-school       3
                  Some-college    283
Length: 201, dtype: int64

In [7]:
s = group_two_count('Occupation', 'Education')
assert s['Transport-moving']['Doctorate'] == 1
assert s['Adm-clerical']['10th'] == 38

## Question 3

Write code to perform a differencing attack to determine Brenn McNeely's age using only the `mean` aggregation function over large groups.

In [8]:
def get_brenns_age():
     return adult['Age'].sum() - adult[adult['Name'] != 'Brenn McNeely']['Age'].sum()

brenns_age = get_brenns_age()

In [9]:
assert brenns_age == 38.0

In [10]:
df = adult[['Education', 'Marital Status', 'Target']]
df.head(10)

,Education,Marital Status,Target
0,Bachelors,Never-married,<=50K
1,Bachelors,Married-civ-spouse,<=50K
2,HS-grad,Divorced,<=50K
3,11th,Married-civ-spouse,<=50K
4,Bachelors,Married-civ-spouse,<=50K
5,Masters,Married-civ-spouse,<=50K
6,9th,Married-spouse-absent,<=50K
7,HS-grad,Married-civ-spouse,>50K
8,Masters,Never-married,>50K
9,Bachelors,Married-civ-spouse,>50K


## Question 4

Does the dataset `df` satisfy $k$-Anonymity for $k=3$? Why not?

It does not. This is because several groups are smaller than the $k$ value of $3$. These groups include every group but *Bachelors* under *Education* and every group but *Married-civ-spouse* under *Marital Status*

## Question 5

What columns should we designate as quasi-identifiers in the dataset `df`, and why?

All of these columns can be considered quasi-identifiers as none of them directly identify a subject but can fairly easily be used in a linkage attack

## Question 6

Imagine the column `Target` is *not* a quasi-identifier, and we generalize the dataset to achieve $k$-Anonymity for $k=2$ as follows:

- Replace each education level below "HS Grad" with `< HS` and others with `>= HS`
- Replace marital status with `Married` and `Not Married`
- Delete rows as required to achieve $k$-anonymity for $k=2$

For which rows is a homogeneity attack possible, and why?

A homogeneity attack can be used to determine that anyone in the dataset with an education level below "HS Grad" (rows 3 and 6) makes at most $50k annually. This is because all *Education* values of `< HS` are in rows that also have *Target* values of `<= 50k`.

## Question 7

Write code to generalize the `Zip` column in the adult dataset.

In [11]:
def generalize_zip(zip, l):
    # Use modulo to get trailing digits and subtract from original
    return zip - (zip % (10 ** l))

In [12]:
# Test cases
assert generalize_zip(47401, 0) == 47401
assert generalize_zip(47401, 1) == 47400
assert generalize_zip(47401, 2) == 47400
assert generalize_zip(47401, 3) == 47000
assert generalize_zip(47401, 4) == 40000
assert generalize_zip(47401, 5) == 0